In [67]:
!pip install plot_keras_history
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from plot_keras_history import show_history
import matplotlib.pyplot as plt
import numpy as np
import h5py


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [68]:
import mlflow

remote_server_uri = "http://127.0.0.1:8080"  # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment("Assignment 5 - MNIST")

2024/04/21 15:30:30 INFO mlflow.tracking.fluent: Experiment with name 'Assignment 5 - MNIST' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/665051419602711718', creation_time=1713693630694, experiment_id='665051419602711718', last_update_time=1713693630694, lifecycle_stage='active', name='Assignment 5 - MNIST', tags={}>

In [69]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()
num_classes = 10
x_train = X_train.reshape(60000, 784)
x_test = X_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape, 'train input samples')
print(x_test.shape, 'test input samples')

(60000, 784) train input samples
(10000, 784) test input samples


In [70]:
y_train = keras.utils.to_categorical(Y_train, num_classes)
y_test = keras.utils.to_categorical(Y_test, num_classes)
print(y_train.shape, 'train output samples')
print(y_test.shape, 'test output samples')

(60000, 10) train output samples
(10000, 10) test output samples


In [71]:
mlflow.autolog(disable = True)
with mlflow.start_run(run_name = 'model1 with manual logging'):
    
    # let's try a basic neural network for digit classification
    model = keras.Sequential()
    model.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
    model.add(layers.Dense(20, activation='sigmoid'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    epochs = 10
    history = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))

    train_loss, train_acc = model.evaluate(x_train, y_train)
    test_loss, test_acc = model.evaluate(x_test, y_test)
    
    mlflow.log_param('epochs', epochs)
    
    mlflow.log_metric('train_loss', train_loss)
    mlflow.log_metric('train_acc', train_acc)

    mlflow.log_metric('test_loss', test_loss)
    mlflow.log_metric('test_acc', test_acc)

    mlflow.keras.log_model(model, 'model1-no-autolog')
    

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5873 - loss: 1.5301 - val_accuracy: 0.8746 - val_loss: 0.4920
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8856 - loss: 0.4332 - val_accuracy: 0.9144 - val_loss: 0.3176
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9166 - loss: 0.2991 - val_accuracy: 0.9265 - val_loss: 0.2602
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9291 - loss: 0.2495 - val_accuracy: 0.9328 - val_loss: 0.2331
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9363 - loss: 0.2225 - val_accuracy: 0.9364 - val_loss: 0.2193
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9436 - loss: 0.1966 - val_accuracy: 0.9410 - val_loss: 0.2019
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9458 - loss: 0.1863 - val_accuracy: 0.9458 - val_loss: 0.1886
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9505 - loss: 0.1753 - 

2024/04/21 15:30:57 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


In [72]:
mlflow.autolog()
with mlflow.start_run(run_name = 'model1 with autologging'):
    
    # let's try a basic neural network for digit classification
    model = keras.Sequential()
    model.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
    model.add(layers.Dense(20, activation='sigmoid'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

    train_loss, train_acc = model.evaluate(x_train, y_train)
    test_loss, test_acc = model.evaluate(x_test, y_test)

    mlflow.log_metric('test_loss', test_loss)
    mlflow.log_metric('test_acc', test_acc)

2024/04/21 15:31:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2024/04/21 15:31:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.6092 - loss: 1.5135 - val_accuracy: 0.9020 - val_loss: 0.3928
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9058 - loss: 0.3574 - val_accuracy: 0.9205 - val_loss: 0.2799
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9253 - loss: 0.2650 - val_accuracy: 0.9326 - val_loss: 0.2382
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9337 - loss: 0.2292 - val_accuracy: 0.9370 - val_loss: 0.2160
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9397 - loss: 0.2085 - val_accuracy: 0.9423 - val_loss: 0.2022
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9441 - loss: 0.1883 - val_accuracy: 0.9437 - val_loss: 0.1963
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9500 - loss: 0.1729 - val_accuracy: 0.9461 - val_loss: 0.1846
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9515 - loss: 0.1666 - 

In [73]:
mlflow.autolog()
with mlflow.start_run(run_name = 'model2') as parent_run:
    
    # Let's try with a slightly bigger model with more parameters.
    with mlflow.start_run(run_name= 'basic', nested=True) as child_run:
        model2 = keras.Sequential()
        model2.add(layers.Dense(256, activation='sigmoid', input_shape=(784,)))
        model2.add(layers.Dense(128, activation='sigmoid'))
        model2.add(layers.Dense(10, activation='softmax'))
        
        model2.compile(loss='categorical_crossentropy', metrics=['accuracy'])
        history = model2.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

        train_loss, train_acc = model2.evaluate(x_train, y_train)
        test_loss, test_acc = model2.evaluate(x_test, y_test)
    
        mlflow.log_metric('test_loss', test_loss)
        mlflow.log_metric('test_acc', test_acc)
        
    # let's try adding kernel regularization to the mix.    
    with mlflow.start_run(run_name= 'with kernel regularization', nested=True) as child_run:
        model_r = keras.Sequential()
        model_r.add(layers.Dense(256, activation='sigmoid', input_shape=(784,), kernel_regularizer=regularizers.L2(0.01)))
        model_r.add(layers.Dense(128, activation='sigmoid', kernel_regularizer=regularizers.L2(0.01)))
        model_r.add(layers.Dense(10, activation='softmax'))

        model_r.compile(loss='categorical_crossentropy', metrics=['accuracy'])
        history_r = model_r.fit(x_train, y_train, epochs=50, steps_per_epoch=50, validation_data=(x_test, y_test))
                
        train_loss, train_acc = model_r.evaluate(x_train, y_train)
        test_loss, test_acc = model_r.evaluate(x_test, y_test)
        
        mlflow.log_metric('test_loss', test_loss)
        mlflow.log_metric('test_acc', test_acc)
        
    # Adding dropping layers instead
    with mlflow.start_run(run_name= 'with dropping layers', nested=True) as child_run:
        model_rd = keras.Sequential()
        model_rd.add(layers.Dense(256, activation='sigmoid', input_shape=(784,)))
        model_rd.add(Dropout(0.7))
        model_rd.add(layers.Dense(128, activation='sigmoid'))
        model_rd.add(Dropout(0.6))
        model_rd.add(layers.Dense(10, activation='softmax'))

        model_rd.compile(loss='categorical_crossentropy', metrics=['accuracy'])
        history_rd = model_rd.fit(x_train, y_train, epochs=10)

        train_loss, train_acc = model_rd.evaluate(x_train, y_train)
        test_loss, test_acc = model_rd.evaluate(x_test, y_test)

        mlflow.log_metric('test_loss', test_loss)
        mlflow.log_metric('test_acc', test_acc)
        
    # EARLY STOPPING METHOD
    with mlflow.start_run(run_name= 'with early stopping', nested=True) as child_run:
        model_re = keras.Sequential()
        model_re.add(layers.Dense(256, activation='sigmoid', input_shape=(784,)))
        model_re.add(layers.Dense(128, activation='sigmoid'))
        model_re.add(layers.Dense(10, activation='softmax'))

        model_re.compile(loss='categorical_crossentropy', metrics=['accuracy'])
        
        # model will get saved at the end of every epoch automatically.
        checkpoint = ModelCheckpoint(r"mnist-epoch-{epoch:02d}.keras")
        history_es = model_re.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[checkpoint])
        
        es = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=2)
        model_re.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[es])

        train_loss, train_acc = model_re.evaluate(x_train, y_train)
        test_loss, test_acc = model_re.evaluate(x_test, y_test)

2024/04/21 15:31:34 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2024/04/21 15:31:34 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8096 - loss: 0.7020 - val_accuracy: 0.9315 - val_loss: 0.2183
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9438 - loss: 0.1879 - val_accuracy: 0.9544 - val_loss: 0.1440
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9620 - loss: 0.1244 - val_accuracy: 0.9645 - val_loss: 0.1118
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9720 - loss: 0.0908 - val_accuracy: 0.9696 - val_loss: 0.0922
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9785 - loss: 0.0717 - val_accuracy: 0.9721 - val_loss: 0.0869
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9822 - loss: 0.0562 - val_accuracy: 0.9745 - val_loss: 0.0801
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9850 - loss: 0.0476 - val_accuracy: 0.9743 - val_loss: 0.0862
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9883 - loss: 0.0382 - 

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.2649 - loss: 5.7403 - val_accuracy: 0.5432 - val_loss: 2.8573
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5432 - val_loss: 2.8573
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5403 - loss: 2.5745 - val_accuracy: 0.6172 - val_loss: 2.0720
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6172 - val_loss: 2.0720
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6102 - loss: 2.0151 - val_accuracy: 0.6436 - val_loss: 1.8685
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6436 - val_loss: 1.8685
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6497 - loss: 1.8370 - val_accuracy: 0.7197 - val_loss: 1.7554
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+0

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5582 - loss: 1.3065
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8623 - loss: 0.4624
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8894 - loss: 0.3903
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9039 - loss: 0.3452
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9102 - loss: 0.3194
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9143 - loss: 0.3123
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9212 - loss: 0.2970
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9227 - loss: 0.2882
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9260 - loss: 0.2840
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9277 - loss: 0.2721
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9616 - loss: 0.1407
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step -

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8058 - loss: 0.7026 - val_accuracy: 0.9390 - val_loss: 0.2052
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9420 - loss: 0.1947 - val_accuracy: 0.9550 - val_loss: 0.1461
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9597 - loss: 0.1293 - val_accuracy: 0.9652 - val_loss: 0.1065
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9711 - loss: 0.0962 - val_accuracy: 0.9715 - val_loss: 0.0881
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9771 - loss: 0.0759 - val_accuracy: 0.9724 - val_loss: 0.0860
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9814 - loss: 0.0601 - val_accuracy: 0.9769 - val_loss: 0.0734
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9855 - loss: 0.0487 - val_accuracy: 0.9762 - val_loss: 0.0753
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9878 - loss: 0.0404 - 

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9925 - loss: 0.0243 - val_accuracy: 0.9789 - val_loss: 0.0748
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9948 - loss: 0.0182 - val_accuracy: 0.9759 - val_loss: 0.0868
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9952 - loss: 0.0173 - val_accuracy: 0.9773 - val_loss: 0.0814
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9956 - loss: 0.0143
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9735 - loss: 0.0967


In [74]:
mlflow.autolog()
with mlflow.start_run(run_name = 'model3') as parent_run:
    
    # Using LEarning rates now.
    with mlflow.start_run(run_name= 'basic', nested=True) as child_run:
        model3 = keras.Sequential()
        model3.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
        model3.add(layers.Dense(10, activation='sigmoid'))
        model3.add(layers.Dense(10, activation='softmax'))

        opt_new = keras.optimizers.SGD(learning_rate=10.0)
        model3.compile(optimizer=opt_new, loss='categorical_crossentropy', metrics=['accuracy'])
        model3.fit(x_train, y_train, epochs=20)

        train_loss, train_acc = model3.evaluate(x_train, y_train)
        test_loss, test_acc = model3.evaluate(x_test, y_test)

        mlflow.log_metric('test_loss', test_loss)
        mlflow.log_metric('test_acc', test_acc)
        
    # too low learning rate
    with mlflow.start_run(run_name= 'too low learning rate', nested=True) as child_run:
        model3 = keras.Sequential()
        model3.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
        model3.add(layers.Dense(10, activation='sigmoid'))
        model3.add(layers.Dense(10, activation='softmax'))

        opt_new = keras.optimizers.SGD(learning_rate=10.0)
        model3.compile(optimizer=opt_new, loss='categorical_crossentropy', metrics=['accuracy'])
        model3.fit(x_train, y_train, epochs=20)

        train_loss, train_acc = model3.evaluate(x_train, y_train)
        test_loss, test_acc = model3.evaluate(x_test, y_test)

        mlflow.log_metric('test_loss', test_loss)
        mlflow.log_metric('test_acc', test_acc)
        
    # optimal learning rate
    with mlflow.start_run(run_name= 'optimal learning rate', nested=True) as child_run:
        model3 = keras.Sequential()
        model3.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
        model3.add(layers.Dense(10, activation='sigmoid'))
        model3.add(layers.Dense(10, activation='softmax'))

        opt_new = keras.optimizers.SGD(learning_rate=.01)
        model3.compile(optimizer=opt_new, loss='categorical_crossentropy', metrics=['accuracy'])
        model3.fit(x_train, y_train, epochs=20)

        train_loss, train_acc = model3.evaluate(x_train, y_train)
        test_loss, test_acc = model3.evaluate(x_test, y_test)

        mlflow.log_metric('test_loss', test_loss)
        mlflow.log_metric('test_acc', test_acc)
        
    # optimal learning rate with momentum
    with mlflow.start_run(run_name= 'optimal learning rate with momentum', nested=True) as child_run:
        model3 = keras.Sequential()
        model3.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
        model3.add(layers.Dense(10, activation='sigmoid'))
        model3.add(layers.Dense(10, activation='softmax'))

        opt_new = keras.optimizers.SGD(learning_rate=.01, momentum=0.5)
        model3.compile(optimizer=opt_new, loss='categorical_crossentropy', metrics=['accuracy'])
        model3.fit(x_train, y_train, epochs=20)

        train_loss, train_acc = model3.evaluate(x_train, y_train)
        test_loss, test_acc = model3.evaluate(x_test, y_test)

         

2024/04/21 15:35:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2024/04/21 15:35:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.


Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 875us/step - accuracy: 0.1876 - loss: 2.2659
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 890us/step - accuracy: 0.4121 - loss: 1.6685
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 866us/step - accuracy: 0.4977 - loss: 1.5098
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 875us/step - accuracy: 0.5361 - loss: 1.4919
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 895us/step - accuracy: 0.5819 - loss: 1.3587
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 880us/step - accuracy: 0.5967 - loss: 1.3096
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 967us/step - accuracy: 0.6118 - loss: 1.2727
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 967us/step - accuracy: 0.7117 - loss: 1.0521
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 887us/step - accuracy: 0.7188 - loss: 1.0072
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 894us/step - accuracy: 0.7372 - loss: 0.9368
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 891us/step - accuracy: 0.7468 - loss: 0.8884
Epoch 12

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 922us/step - accuracy: 0.2016 - loss: 2.2742
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 878us/step - accuracy: 0.6433 - loss: 1.1210
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 870us/step - accuracy: 0.7882 - loss: 0.7827
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 880us/step - accuracy: 0.8155 - loss: 0.7002
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 924us/step - accuracy: 0.8212 - loss: 0.6698
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 883us/step - accuracy: 0.8244 - loss: 0.6461
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 920us/step - accuracy: 0.8357 - loss: 0.5992
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 889us/step - accuracy: 0.8348 - loss: 0.5990
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 897us/step - accuracy: 0.8311 - loss: 0.5973
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 867us/step - accuracy: 0.8669 - loss: 0.5217
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 930us/step - accuracy: 0.8605 - loss: 0.5153
Epoch 12

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1784 - loss: 2.2928
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 896us/step - accuracy: 0.4599 - loss: 2.0982
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 879us/step - accuracy: 0.5308 - loss: 1.7831
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 870us/step - accuracy: 0.6050 - loss: 1.4642
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 864us/step - accuracy: 0.6841 - loss: 1.2177
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 833us/step - accuracy: 0.7337 - loss: 1.0428
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 880us/step - accuracy: 0.7682 - loss: 0.9167
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 875us/step - accuracy: 0.7960 - loss: 0.8266
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 883us/step - accuracy: 0.8133 - loss: 0.7505
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 931us/step - accuracy: 0.8293 - loss: 0.6879
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 907us/step - accuracy: 0.8427 - loss: 0.6380
Epoch 12/2

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 904us/step - accuracy: 0.2305 - loss: 2.2537
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 904us/step - accuracy: 0.5761 - loss: 1.7752
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 924us/step - accuracy: 0.6635 - loss: 1.2959
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 904us/step - accuracy: 0.7398 - loss: 0.9717
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 910us/step - accuracy: 0.7999 - loss: 0.7589
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 941us/step - accuracy: 0.8426 - loss: 0.6328
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 907us/step - accuracy: 0.8662 - loss: 0.5455
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 989us/step - accuracy: 0.8811 - loss: 0.4757
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 943us/step - accuracy: 0.8938 - loss: 0.4285
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 913us/step - accuracy: 0.8990 - loss: 0.3988
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 913us/step - accuracy: 0.9030 - loss: 0.3763
Epoch 12

In [75]:
mlflow.autolog()
with mlflow.start_run(run_name = 'model4'):
    # Mini-batch SGD
    # the default minibatch size is 32 unlike 1.
    model4 = keras.Sequential()
    model4.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
    model4.add(layers.Dense(10, activation='sigmoid'))
    model4.add(layers.Dense(10, activation='softmax'))
    
    opt_new = keras.optimizers.SGD(learning_rate=.01, momentum=0.5)
    model4.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    model4.fit(x_train, y_train, batch_size=512, epochs=10)

    train_loss, train_acc = model4.evaluate(x_train, y_train)
    test_loss, test_acc = model4.evaluate(x_test, y_test)

    mlflow.log_metric('test_loss', test_loss)
    mlflow.log_metric('test_acc', test_acc)

2024/04/21 15:37:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2024/04/21 15:37:57 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.


Epoch 1/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2634 - loss: 2.2232
Epoch 2/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5749 - loss: 1.8484
Epoch 3/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6854 - loss: 1.5737
Epoch 4/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7478 - loss: 1.3232
Epoch 5/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7841 - loss: 1.1041
Epoch 6/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8188 - loss: 0.9250
Epoch 7/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8444 - loss: 0.7823
Epoch 8/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8650 - loss: 0.6664
Epoch 9/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8796 - loss: 0.5767
Epoch 10/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8890 - loss: 0.5093
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 825us/step - accuracy: 0.8982 - loss: 0.4653
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step - accuracy: 0.877

In [76]:
mlflow.autolog()
with mlflow.start_run(run_name = 'All 10 in same run') as grandparent_run:
    
    #model 1
    with mlflow.start_run(run_name = 'model1 with autologging', nested = True) as child_run:
        
        # let's try a basic neural network for digit classification
        model = keras.Sequential()
        model.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
        model.add(layers.Dense(20, activation='sigmoid'))
        model.add(layers.Dense(10, activation='softmax'))
    
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
        history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
    
        train_loss, train_acc = model.evaluate(x_train, y_train)
        test_loss, test_acc = model.evaluate(x_test, y_test)

        mlflow.log_metric('test_loss', test_loss)
        mlflow.log_metric('test_acc', test_acc)

    #model 2
    with mlflow.start_run(run_name = 'model2', nested = True) as parent_run:
        
        # Let's try with a slightly bigger model with more parameters.
        with mlflow.start_run(run_name= 'basic', nested=True) as child_run:
            model2 = keras.Sequential()
            model2.add(layers.Dense(256, activation='sigmoid', input_shape=(784,)))
            model2.add(layers.Dense(128, activation='sigmoid'))
            model2.add(layers.Dense(10, activation='softmax'))
            
            model2.compile(loss='categorical_crossentropy', metrics=['accuracy'])
            history = model2.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
    
            train_loss, train_acc = model2.evaluate(x_train, y_train)
            test_loss, test_acc = model2.evaluate(x_test, y_test)
        
            mlflow.log_metric('test_loss', test_loss)
            mlflow.log_metric('test_acc', test_acc)
        
        # let's try adding kernel regularization to the mix.    
        with mlflow.start_run(run_name= 'with kernel regularization', nested=True) as child_run:
            model_r = keras.Sequential()
            model_r.add(layers.Dense(256, activation='sigmoid', input_shape=(784,), kernel_regularizer=regularizers.L2(0.01)))
            model_r.add(layers.Dense(128, activation='sigmoid', kernel_regularizer=regularizers.L2(0.01)))
            model_r.add(layers.Dense(10, activation='softmax'))
    
            model_r.compile(loss='categorical_crossentropy', metrics=['accuracy'])
            history_r = model_r.fit(x_train, y_train, epochs=50, steps_per_epoch=50, validation_data=(x_test, y_test))
                    
            train_loss, train_acc = model_r.evaluate(x_train, y_train)
            test_loss, test_acc = model_r.evaluate(x_test, y_test)

            mlflow.log_metric('test_loss', test_loss)
            mlflow.log_metric('test_acc', test_acc)
        
        # Adding dropping layers instead
        with mlflow.start_run(run_name= 'with dropping layers', nested=True) as child_run:
            model_rd = keras.Sequential()
            model_rd.add(layers.Dense(256, activation='sigmoid', input_shape=(784,)))
            model_rd.add(Dropout(0.7))
            model_rd.add(layers.Dense(128, activation='sigmoid'))
            model_rd.add(Dropout(0.6))
            model_rd.add(layers.Dense(10, activation='softmax'))
    
            model_rd.compile(loss='categorical_crossentropy', metrics=['accuracy'])
            history_rd = model_rd.fit(x_train, y_train, epochs=10)
    
            train_loss, train_acc = model_rd.evaluate(x_train, y_train)
            test_loss, test_acc = model_rd.evaluate(x_test, y_test)

            mlflow.log_metric('test_loss', test_loss)
            mlflow.log_metric('test_acc', test_acc)
            
        # EARLY STOPPING METHOD
        with mlflow.start_run(run_name= 'with early stopping', nested=True) as child_run:
            model_re = keras.Sequential()
            model_re.add(layers.Dense(256, activation='sigmoid', input_shape=(784,)))
            model_re.add(layers.Dense(128, activation='sigmoid'))
            model_re.add(layers.Dense(10, activation='softmax'))
    
            model_re.compile(loss='categorical_crossentropy', metrics=['accuracy'])
            
            # model will get saved at the end of every epoch automatically.
            checkpoint = ModelCheckpoint(r"mnist-epoch-{epoch:02d}.keras")
            history_es = model_re.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[checkpoint])
            
            es = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=2)
            model_re.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[es])
    
            train_loss, train_acc = model_re.evaluate(x_train, y_train)
            test_loss, test_acc = model_re.evaluate(x_test, y_test)

            mlflow.log_metric('test_loss', test_loss)
            mlflow.log_metric('test_acc', test_acc)
    
    #model 3
    with mlflow.start_run(run_name = 'model3', nested=True) as parent_run:
        
        # Using LEarning rates now.
        with mlflow.start_run(run_name= 'basic', nested=True) as child_run:
            model3 = keras.Sequential()
            model3.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
            model3.add(layers.Dense(10, activation='sigmoid'))
            model3.add(layers.Dense(10, activation='softmax'))
    
            opt_new = keras.optimizers.SGD(learning_rate=10.0)
            model3.compile(optimizer=opt_new, loss='categorical_crossentropy', metrics=['accuracy'])
            model3.fit(x_train, y_train, epochs=20)
    
            train_loss, train_acc = model3.evaluate(x_train, y_train)
            test_loss, test_acc = model3.evaluate(x_test, y_test)
            
            mlflow.log_metric('test_loss', test_loss)
            mlflow.log_metric('test_acc', test_acc)
            
        # too low learning rate
        with mlflow.start_run(run_name= 'too low learning rate', nested=True) as child_run:
            model3 = keras.Sequential()
            model3.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
            model3.add(layers.Dense(10, activation='sigmoid'))
            model3.add(layers.Dense(10, activation='softmax'))
    
            opt_new = keras.optimizers.SGD(learning_rate=10.0)
            model3.compile(optimizer=opt_new, loss='categorical_crossentropy', metrics=['accuracy'])
            model3.fit(x_train, y_train, epochs=20)
    
            train_loss, train_acc = model3.evaluate(x_train, y_train)
            test_loss, test_acc = model3.evaluate(x_test, y_test)

            mlflow.log_metric('test_loss', test_loss)
            mlflow.log_metric('test_acc', test_acc)
            
        # optimal learning rate
        with mlflow.start_run(run_name= 'optimal learning rate', nested=True) as child_run:
            model3 = keras.Sequential()
            model3.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
            model3.add(layers.Dense(10, activation='sigmoid'))
            model3.add(layers.Dense(10, activation='softmax'))
    
            opt_new = keras.optimizers.SGD(learning_rate=.01)
            model3.compile(optimizer=opt_new, loss='categorical_crossentropy', metrics=['accuracy'])
            model3.fit(x_train, y_train, epochs=20)
    
            train_loss, train_acc = model3.evaluate(x_train, y_train)
            test_loss, test_acc = model3.evaluate(x_test, y_test)

            mlflow.log_metric('test_loss', test_loss)
            mlflow.log_metric('test_acc', test_acc)
            
        # optimal learning rate with momentum
        with mlflow.start_run(run_name= 'optimal learning rate with momentum', nested=True) as child_run:
            model3 = keras.Sequential()
            model3.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
            model3.add(layers.Dense(10, activation='sigmoid'))
            model3.add(layers.Dense(10, activation='softmax'))
    
            opt_new = keras.optimizers.SGD(learning_rate=.01, momentum=0.5)
            model3.compile(optimizer=opt_new, loss='categorical_crossentropy', metrics=['accuracy'])
            model3.fit(x_train, y_train, epochs=20)
    
            train_loss, train_acc = model3.evaluate(x_train, y_train)
            test_loss, test_acc = model3.evaluate(x_test, y_test)

            mlflow.log_metric('test_loss', test_loss)
            mlflow.log_metric('test_acc', test_acc)
    
    #model 4
    with mlflow.start_run(run_name = 'model4', nested = True) as child_run:
        # Mini-batch SGD
        # the default minibatch size is 32 unlike 1.
        model4 = keras.Sequential()
        model4.add(layers.Dense(20, activation='sigmoid', input_shape=(784,)))
        model4.add(layers.Dense(10, activation='sigmoid'))
        model4.add(layers.Dense(10, activation='softmax'))
        
        opt_new = keras.optimizers.SGD(learning_rate=.01, momentum=0.5)
        model4.compile(loss='categorical_crossentropy', metrics=['accuracy'])
        model4.fit(x_train, y_train, batch_size=512, epochs=10)
    
        train_loss, train_acc = model4.evaluate(x_train, y_train)
        test_loss, test_acc = model4.evaluate(x_test, y_test)

        mlflow.log_metric('test_loss', test_loss)
        mlflow.log_metric('test_acc', test_acc)

2024/04/21 15:38:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2024/04/21 15:38:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5978 - loss: 1.4634 - val_accuracy: 0.8827 - val_loss: 0.4584
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8887 - loss: 0.4185 - val_accuracy: 0.9114 - val_loss: 0.3091
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9177 - loss: 0.2918 - val_accuracy: 0.9237 - val_loss: 0.2566
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9288 - loss: 0.2482 - val_accuracy: 0.9340 - val_loss: 0.2232
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9364 - loss: 0.2189 - val_accuracy: 0.9388 - val_loss: 0.2089
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9419 - loss: 0.1993 - val_accuracy: 0.9413 - val_loss: 0.1924
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9482 - loss: 0.1819 - val_accuracy: 0.9455 - val_loss: 0.1824
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9514 - loss: 0.1683 - 

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8050 - loss: 0.7053 - val_accuracy: 0.9359 - val_loss: 0.2067
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9439 - loss: 0.1880 - val_accuracy: 0.9566 - val_loss: 0.1359
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9608 - loss: 0.1302 - val_accuracy: 0.9641 - val_loss: 0.1157
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9717 - loss: 0.0924 - val_accuracy: 0.9696 - val_loss: 0.0925
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9774 - loss: 0.0736 - val_accuracy: 0.9691 - val_loss: 0.0969
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9822 - loss: 0.0594 - val_accuracy: 0.9728 - val_loss: 0.0848
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9841 - loss: 0.0496 - val_accuracy: 0.9777 - val_loss: 0.0750
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9873 - loss: 0.0417 - 

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.3142 - loss: 5.7398 - val_accuracy: 0.5743 - val_loss: 2.8419
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5743 - val_loss: 2.8419
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5540 - loss: 2.5619 - val_accuracy: 0.6506 - val_loss: 2.0333
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6506 - val_loss: 2.0333
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6261 - loss: 1.9896 - val_accuracy: 0.6525 - val_loss: 1.8292
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6525 - val_loss: 1.8292
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6731 - loss: 1.8215 - val_accuracy: 0.7213 - val_loss: 1.7293
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5546 - loss: 1.3160
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8610 - loss: 0.4783
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8897 - loss: 0.3894
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8995 - loss: 0.3506
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9107 - loss: 0.3243
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9133 - loss: 0.3152
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9195 - loss: 0.3011
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9242 - loss: 0.2863
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9257 - loss: 0.2780
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9291 - loss: 0.2677
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9628 - loss: 0.1381
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step -

...

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1960 - loss: 2.2823  
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 898us/step - accuracy: 0.7036 - loss: 1.0111
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 963us/step - accuracy: 0.7904 - loss: 0.7704
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 918us/step - accuracy: 0.8241 - loss: 0.6768
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8431 - loss: 0.5992
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 940us/step - accuracy: 0.8458 - loss: 0.5861
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 927us/step - accuracy: 0.8600 - loss: 0.5480
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 927us/step - accuracy: 0.8567 - loss: 0.5558
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 932us/step - accuracy: 0.8445 - loss: 0.5778
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 924us/step - accuracy: 0.8738 - loss: 0.4963
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 900us/step - accuracy: 0.8708 - loss: 0.5166
Epoch 12/2

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 951us/step - accuracy: 0.3086 - loss: 1.9603
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 908us/step - accuracy: 0.6907 - loss: 1.0403
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 914us/step - accuracy: 0.7510 - loss: 0.9031
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 890us/step - accuracy: 0.7954 - loss: 0.7619
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 876us/step - accuracy: 0.7698 - loss: 0.8610
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 923us/step - accuracy: 0.8107 - loss: 0.7179
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 936us/step - accuracy: 0.8133 - loss: 0.6960
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 973us/step - accuracy: 0.8089 - loss: 0.7111
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 938us/step - accuracy: 0.8254 - loss: 0.6496
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 950us/step - accuracy: 0.8323 - loss: 0.6289
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 933us/step - accuracy: 0.8287 - loss: 0.6445
Epoch 12

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 935us/step - accuracy: 0.1447 - loss: 2.2888
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 899us/step - accuracy: 0.3306 - loss: 2.1659
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 888us/step - accuracy: 0.4966 - loss: 1.9299
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 911us/step - accuracy: 0.6081 - loss: 1.6066
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 896us/step - accuracy: 0.6881 - loss: 1.3134
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 877us/step - accuracy: 0.7356 - loss: 1.1097
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 942us/step - accuracy: 0.7639 - loss: 0.9727
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 980us/step - accuracy: 0.7882 - loss: 0.8653
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 950us/step - accuracy: 0.8043 - loss: 0.7876
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 905us/step - accuracy: 0.8213 - loss: 0.7236
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 949us/step - accuracy: 0.8380 - loss: 0.6653
Epoch 12

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 986us/step - accuracy: 0.2275 - loss: 2.2743
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 896us/step - accuracy: 0.5444 - loss: 1.8143
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 948us/step - accuracy: 0.6734 - loss: 1.2391
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 964us/step - accuracy: 0.7412 - loss: 0.9377
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 951us/step - accuracy: 0.7952 - loss: 0.7623
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 962us/step - accuracy: 0.8310 - loss: 0.6435
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 963us/step - accuracy: 0.8584 - loss: 0.5529
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 986us/step - accuracy: 0.8758 - loss: 0.4851
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 970us/step - accuracy: 0.8849 - loss: 0.4440
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 982us/step - accuracy: 0.8923 - loss: 0.4078
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 944us/step - accuracy: 0.8982 - loss: 0.3799
Epoch 12

Epoch 1/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1945 - loss: 2.2260
Epoch 2/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4731 - loss: 1.8567
Epoch 3/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6101 - loss: 1.5788
Epoch 4/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6994 - loss: 1.3497
Epoch 5/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7693 - loss: 1.1585
Epoch 6/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8201 - loss: 0.9957
Epoch 7/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8501 - loss: 0.8523
Epoch 8/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8691 - loss: 0.7336
Epoch 9/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8763 - loss: 0.6442
Epoch 10/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8823 - loss: 0.5685
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8902 - loss: 0.5131
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - accuracy: 0.8712 